## Fukuro Adaptive Monte Carlo Localization

### A. Download and Importing Dependencies 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [10]:
# AMCLL PARAMETERS
NUM_PARTICLES = 100
MAP_SIZE = (100, 100)

# ROBOT PARAMETERS
MOVE_DISTANCE = 5.0
NOISE_STD = 2.0
robot_position = np.array([20.0, 20.0])

In [11]:
class Particle:
    def __init__(self, x, y, weight=1.0):
        self.x = x
        self.y = y
        self.weight = weight
    def move(self, dx, dy):
        self.x += dx + np.random.normal(0, NOISE_STD) # ONLY FOR SIMULATION
        self.y += dy + np.random.normal(0, NOISE_STD) # ONLY FOR SIMULATION

def particle_generator():
    return [Particle(x=np.random.uniform(0, MAP_SIZE[0]),
                     y=np.random.uniform(0, MAP_SIZE[1]))                    
            for _ in range(NUM_PARTICLES)] # GENERATE N PARTICLES

In [ ]:
# Visualization canvas
fig, ax = plt.subplots(figsize=(6, 6))
sc_particles = ax.scatter([], [], c='blue', label='Particles')
sc_robot = ax.scatter([], [], c='red', s=100, label='Robot')
step_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

def sense(particle, robot_pos): # SELECTING PARTICLES ACCORDING TO EUCLIDIAN DISTANCE AS LIKELIHOOD
    # NORMALIZE DISTANCE
    distance = np.linalg.norm(np.array([particle.x, particle.y]) - robot_pos) 

    # CALCULATING HOW LIKELY THE PARTICLE IS CLOSE WITH TRUE POSITION 
    # THIS CALCULATION USE GAUSSIAN OR NORM DIST PDF FUNCTION
    # SO IT DEPENDS ON THE NOISE STD VALUE 
    return np.exp(-distance**2 / (2 * NOISE_STD**2))

def resample(particles): # PARTICLE WEIGHT BASED RESAMPLING
    weights = np.array([p.weight for p in particles])
    weights += 1e-300 # ADDING EXTREMELY SMALL VALUE TO PREVENT DIVISION BY ZERO
    weights /= np.sum(weights) # NORMALIZE THE WEIGHTS
    